<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/EX20-Multi-factor_Attribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investment Analysis**, Bates, Boyer, Fletcher

# Example Chapter xx: Rick-Factor Exposures and the Performance Spread
In this example we illustrate how to calcualte the contributions of factor exposures to the performance spread in a multi-factor model.

### Imports and Setup

In [ ]:
!curl -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import simple_finance as sf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26346  100 26346    0     0   137k      0 --:--:-- --:--:-- --:--:--  138k


### Load Monthly Returns for Portfolio,  Benchmark, and Fama-French 5 Factors
We first load portfolio and benchmark return data from an Excel file and prepare it for analysis. The Fama–French 5 factor dataset already uses a PeriodIndex with monthly frequency. To make the portfolio and benchmark data consistent, we convert their date index into the same format:
 * `pd.to_datetime(df.index, format='%Y-%m')` converts the index into proper datetime objects (timestamps at the start of each month).

 * `.to_period('M')` then turns those timestamps into a PeriodIndex with monthly frequency.

This ensures that both datasets use the same type of monthly index, which makes them easy to merge.

In [ ]:
# Load in the data by first specifying the URL where the data can be found
url='https://github.com/boyerb/Investments/raw/master/Examples_3.47.xlsx'
# specify which columns to read
columns_to_read = ['date', 'Portfolio','Benchmark']
df = pd.read_excel(url, sheet_name='PA-3', header=0, usecols=columns_to_read, engine='openpyxl')
df = df.dropna()
df = df.set_index('date').sort_index()
df.index  = pd.to_datetime(df.index, format='%Y-%m').to_period('M')

ff5=sf.get_ff5(start_date=None, end_date=None)

### Merge the Two Datasets

In [ ]:
data = df.join(ff5, how='inner')
print(data.head(5))

         Portfolio  Benchmark  Mkt-RF     SMB     HML     RMW     CMA   RF
date                                                                      
2015-01  -0.005135  -0.023922 -0.0309 -0.0093 -0.0345  0.0158 -0.0164  0.0
2015-02   0.065317   0.007846  0.0614  0.0036 -0.0179 -0.0110 -0.0175  0.0
2015-03  -0.004808   0.016960 -0.0109  0.0308 -0.0038  0.0007 -0.0062  0.0
2015-04  -0.014598   0.005166  0.0060 -0.0301  0.0180  0.0005 -0.0062  0.0
2015-05  -0.002515   0.000142  0.0138  0.0082 -0.0111 -0.0176 -0.0083  0.0


### Create Excess Returns and Define Explanatory Variables

In [ ]:
# Excess returns
data['Port_excess'] = data['Portfolio'] - data['RF']
data['Bench_excess'] = data['Benchmark'] - data['RF']
data['Active_excess'] = data['Portfolio'] - data['Benchmark']   # active return

# Explaantory Variables (Factor Matrix)
factors = data[['Mkt-RF','SMB','HML','RMW','CMA']]

### Regression Attribution Function
The `run_reg_details` function runs an OLS regression of a return series on the Fama–French factors and produces two useful tables:

 * Coefficients table — shows the estimated regression coefficients (alpha, b1, b2, …).

 * Contributions table — shows the contribution of each factor to average return, computed as beta × average factor return, along with alpha and the total.

**How it works:**  
You give the function a return series (portfolio, benchmark, or active), the factor matrix, and a label (name). The function runs an OLS regression, extracts the regression coefficients to build  a  **coefficients table**, and then multiplies each factor’s coefficient by its average return to build a  **contributions table**.

**Inputs you provide:**  

 * `y`: the dependent variable (e.g., portfolio excess return, benchmark excess return, or active excess return).
 * `X`: the independent variables (the factor matrix).
 * `name`: a string label for the regression (e.g., `"Portfolio"`, `"Benchmark"`, `"Active"`).

**What you get back:**

 * `coefs`: a Series with alpha and each regression coefficient (b1, b2, …).
 * `contribs`: a Series with alpha, each bi*avg(Fi), and the total.

In [ ]:
def run_reg_details(y, X, name):
    """
    OLS of y on X (with intercept). Returns:
      - coefs:    Alpha, b1, b2, ...             (as a Series named `name`)
      - contribs: Alpha, b1*avg(F1), ..., Total (as a Series named `name`)
    """
    model = sm.OLS(y, sm.add_constant(X)).fit()
    alpha = model.params["const"]
    betas = model.params.drop("const")

    # Coefficients: Alpha, b1..bK
    coefs = pd.Series({"Alpha": alpha, **{f"b{i}": betas[f] for i, f in enumerate(X.columns, 1)}}, name=name)

    # Contributions: Alpha, b1*avg(F1).., Total
    contrib_items = {"Alpha": alpha}
    contrib_items.update({f"b{i}*avg({f})": betas[f] * X[f].mean() for i, f in enumerate(X.columns, 1)})
    contrib_items["Total"] = sum(contrib_items.values())
    contribs = pd.Series(contrib_items, name=name)

    return coefs, contribs


### Run Factor Regressions and Build Tables
We now run the regression function run_reg_details for the portfolio, benchmark, and active return series. Each regression produces two outputs:

 * Coefficients table: shows alpha and the raw factor betas.
 * Contributions table: shows alpha and each factor’s contribution ($\beta \times$ average factor return).

The first block runs the regression three times: once for portfolio excess returns, once for benchmark excess returns, and once for active (portfolio – benchmark).

The second block combines the results side by side into two summary tables:

 * `coef_table`: the regression coefficients ($\alpha$ and betas) for each regression.  
 * `contrib_table`: annualized contributions (each monthly number is multiplied by 12 to annualize).



In [ ]:
# Run for Portfolio, Benchmark, Active
coefs_port, contrib_port  = run_reg_details(data["Port_excess"],  factors, "Portfolio")
coefs_bench, contrib_bench  = run_reg_details(data["Bench_excess"], factors, "Benchmark")
coefs_active, contrib_active  = run_reg_details(data["Active_excess"], factors, "Active")

# Build tables
coef_table    = pd.concat([coefs_port,  coefs_bench,  coefs_active],  axis=1)



### Print the regression coefficients
The `\n` adds a blank line after printing each table.

In [ ]:
print("Regression Coefficients")
print(coef_table, "\n")

print("Contributions")
print(contrib_active, "\n")

# Show that Active Total equals performance spread (mean Active excess)
mean_excess = pd.Series({
    "Active":    data["Active_excess"].mean(),
}, name="Mean Excess")

print("Performance Spread", data["Active_excess"].mean())

Regression Coefficients
       Portfolio  Benchmark    Active
Alpha   0.001531  -0.000942  0.002473
b1      1.152011   0.397230  0.754782
b2      0.346967   0.587496 -0.240529
b3     -0.616106   0.370690 -0.986795
b4      1.009803   0.399950  0.609853
b5      0.902449   0.054175  0.848274 

Contributions
Alpha             0.002473
b1*avg(Mkt-RF)    0.007412
b2*avg(SMB)       0.000398
b3*avg(HML)       0.001817
b4*avg(RMW)       0.002416
b5*avg(CMA)      -0.000734
Total             0.013781
Name: Active, dtype: float64 

Performance Spread 0.013780866666666666


### Summary
The average performance spread is $1.378\%$ as shown in the last line of the output block above.

*Regression Coefficients*  
The first table above presents the regression coefficients.  Note that the values in the **Active** column are equal to the **Portfolio** value minus the **Benchmark** value.  We get these differences by simply regressing the active portfolio return on the five factrors. The portfolio has a positive alpha of $15.31$ basis points per month, and the benchmark has a negative alpha of $-9.42$ basis points per month, implying an alpha spread of $24.73$ basis points per month.The portfolio has greater exposure to the market factor (b1), the RMW factor (b2) and the CMA factor (b5), and lower exposure to the SMB (b2) and HML (b3) factors.

*Contributions to Performance Spread*   
The contributions table decomposes the **performance spread** into six parts: alpha and the contributions from exposure to each of the five risk factors. Greater exposure to the market factor is adding a whopping 74.12 basis points to the performance spread. The next biggest contributor is RMW, the exposure to which is adding another $24.16$ basis points. The bottom line of the contributions table adds up all the component effects, to show that the sum is equal to the performance spread, $1.378\%.$   